### Random Notes Here

Train "12th man" RL agent to become a perfect tackler. Metric is deviance from this path.

GHOST IN THE MACHINE; concept of "ghosts" like in mario kart and other video games

Model from different positions - i.e. agent from defensive lineman initial starting position
    remove impossible scenarios based on distribution of distance covered and speed for example

TODO : adjust window to match action space?
Define "a tackle" using empirical data - how does the agent know when it's completed its task? Part of the reward function.

# Creating an American Football Reinforcement Learning Enviornment 

In [9]:
import pygame
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import nfl_data_py as nfl

In [10]:
from FootballPlay import FootballPlay
from PreprocessTrackingData import PreprocessTrackingData

In [11]:
# Turn off pandas column limits
pd.set_option('display.max_columns', None)

In [12]:
# Read in the data
tracking = pd.read_csv("data/tracking_week_1.csv")
plays = pd.read_csv("data/plays.csv")

In [13]:
tracking.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


In [14]:
# from gymnasium.envs.registration import register

# register(
#      id="FootballPlay-v0",
#      entry_point=FootballPlay,
#      max_episode_steps=300,
# )

In [15]:
gameId = 2022090800
playId = 56

# Assuming df is your DataFrame containing the tracking data
preprocessor = PreprocessTrackingData(tracking, plays, gameId, playId)
play_data = preprocessor.get_processed_data()

# Find the maximum frame ID
max_frames = play_data['frameId'].max()

# Now, create an instance of FootballPlay with this max_frame_id
env = FootballPlay(play_data, render_mode='human', max_frames=max_frames)

In [16]:
for _ in range(1000):
    observation, info = env.reset()
    done = False

    while not done:
        action = env.action_space.sample()  # Sample an action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            done = True  # Mark the episode as done

env.close()

KeyboardInterrupt: 